In [ ]:
pip install streamlit transformers huggingface_hub peft torch datetime

In [ ]:
%%writefile my_file.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from peft import PeftModel, PeftConfig
from datetime import datetime
import torch

# Set up Hugging Face token directly
HF_TOKEN = ""  # Replace with your Hugging Face token
MODEL_NAME = "iyashnayi/SocioLens-llama-3.2-3B"
login(token=HF_TOKEN)

# Configure page layout
st.set_page_config(page_title="SocioLens", layout="wide")

# Custom styling
st.markdown("""
    <style>
        /* Sidebar styling */
        section[data-testid="stSidebar"] {
            background-color: #1C2526;
            color: #FFFFFF;
            display: flex;
            flex-direction: column;
        }
        /* Main pane styling */
        .main {
            background-color: #000000;
            font-family: 'Inter', sans-serif;
            color: #FFFFFF;
            position: relative;
            min-height: 100vh;
        }
        /* Header container */
        .header-container {
            display: flex;
            justify-content: center;
            align-items: center;
            margin-top: 2rem;
            margin-bottom: 1rem;
        }
        /* Title styling */
        .header-logo {
            font-size: 3rem;
            font-weight: bold;
            color: #FF9F1C;
            font-family: 'Inter', sans-serif;
            letter-spacing: -1px;
            margin-bottom: 0.5rem;
        }
        /* Subtitle styling */
        .header-subtitle {
            font-size: 1.2rem;
            color: #FF9F1C;
            font-style: italic;
            text-align: center;
            opacity: 0;
            animation: fadeIn 1.5s forwards;
            animation-delay: 0.5s;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        /* Button styling */
        .stButton>button {
            background-color: #444444;
            color: white;
            border: none;
            border-radius: 6px;
            padding: 0.5rem 1rem;
            font-weight: 500;
        }
        .stButton>button:hover {
            background-color: #555555;
        }
        /* Chat message styling */
        .stChatMessage {
            background-color: #2D2D2D;
            border-radius: 10px;
            padding: 15px;
            margin-bottom: 15px;
            border-left: 3px solid #FF9F1C;
            color: #FFFFFF;
        }
        .chat-area {
            padding-bottom: 80px;
        }
        .sidebar-content {
            flex: 1;
        }
        .sidebar-upload {
            margin-top: auto;
            padding-bottom: 20px;
            border-top: 1px solid #444444;
            padding-top: 15px;
        }
        .stSidebar div[data-testid="stFileUploader"] {
            background-color: #2D2D2D;
            border-radius: 8px;
            padding: 10px;
            margin-top: 10px;
        }
        .stSidebar div[data-testid="stFileUploader"] label {
            color: #FFFFFF;
            font-weight: bold;
        }
        .upload-button {
            width: 100%;
            background-color: #444444;
            color: white;
            border: none;
            border-radius: 6px;
            padding: 8px;
            font-weight: 500;
            margin-top: 5px;
            cursor: pointer;
            text-align: center;
        }
        .sidebar-header {
            color: #FFFFFF;
            font-weight: 600;
            margin-top: 1rem;
        }
        .element-container:has(footer) {display: none;}
        #MainMenu {visibility: hidden;}
        header {visibility: hidden;}
    </style>
""", unsafe_allow_html=True)

# Header
st.markdown("""
    <div class="header-container">
        <div>
            <div class="header-logo">💬 SocioLens</div>
            <div class="header-subtitle">Your Public Policy Analysis Assistant</div>
        </div>
    </div>
""", unsafe_allow_html=True)

st.markdown("<style>h1 {display: none;}</style>", unsafe_allow_html=True)

# Define policy analysis & greetings prompt templates
prompt_templates = {
    "analysis": """You are SocioLens — a data-driven and experienced public policy analyst specializing in social impact, governance, and economic equity.

Analyze the following policy issue:
{query}

Approach:
1. **Executive Summary** – Provide a 1-2 sentence summary of the issue and key insight.
2. **Context** – Briefly situate the issue in historical, social, or economic terms.
3. **Key Factors** – Identify driving forces, affected populations, and relevant data indicators.
4. **Stakeholder Impact** – Who gains, who bears the cost, and what power dynamics exist?
5. **Evidence-Based Recommendations** – Provide 2-3 clear policy options with pros/cons.
6. **Implementation Notes** – Mention feasibility, barriers, or trade-offs if applicable.

Keep language clear, neutral, and grounded in data. Highlight uncertainties or assumptions explicitly.""",

    "comparative": """You are SocioLens — a veteran public policy analyst skilled in comparative frameworks and stakeholder-centered evaluation.

Compare the following policy approaches:
{query}

Structure your response as follows:
1. **Policy Snapshot** – Briefly outline each approach in 1–2 sentences.
2. **Evaluation Criteria** – Use 3-4 key axes: effectiveness, cost, equity, and implementation risk.
3. **Stakeholder Analysis** – Discuss who benefits/loses under each option.
4. **Evidence** – Reference real-world data, cases, or trends if possible.
5. **Conclusion** – State which policy fits best in what scenario, and note any hybrid strategies.

Maintain neutrality, present pros/cons clearly, and surface any political or ethical tensions.""",

    "forecast": """You are SocioLens — an experienced policy analyst focused on scenario modeling and systemic forecasting.

Forecast the outcomes of the following policy change:
{query}

Organize your response as follows:
1. **Short-Term Outlook (1–2 yrs)** – Immediate implementation effects and likely reactions.
2. **Medium-Term Outlook (3–5 yrs)** – Shifts in behavior, cost structures, or demographics.
3. **Long-Term Effects (5+ yrs)** – Institutional, environmental, or generational consequences.
4. **Unintended Consequences** – Identify risks, externalities, or vulnerable populations.
5. **Confidence Levels** – Indicate high/medium/low certainty in predictions.
6. **Monitoring Recommendations** – Suggest what metrics or signals should be tracked.

Use a tone that is cautious, analytic, and contingency-aware. Point out where assumptions drive outcomes.""",

    "greeting": """You are SocioLens — a friendly and welcoming public policy assistant.

The user says: "{query}"

Respond with a warm greeting and ask how you can assist, without any policy analysis."""
}

@st.cache_resource
def load_model():
    peft_model_id = MODEL_NAME
    config = PeftConfig.from_pretrained(peft_model_id)
    base_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(base_model, peft_model_id)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    return model, tokenizer

model, tokenizer = load_model()

# Session state setup
if "messages" not in st.session_state:
    st.session_state.messages = []
if "chat_history" not in st.session_state:
    st.session_state.chat_history = {}
if "uploaded_file" not in st.session_state:
    st.session_state.uploaded_file = None
if "analysis_mode" not in st.session_state:
    st.session_state.analysis_mode = "analysis"

# Sidebar UI
with st.sidebar:
    st.markdown('<div class="sidebar-content">', unsafe_allow_html=True)
    st.markdown('<h3 class="sidebar-header">📊 Analysis Mode</h3>', unsafe_allow_html=True)
    analysis_mode = st.radio(
        "Select analysis type:",
        options=["General Analysis", "Comparative Analysis", "Impact Forecast"],
        index=0,
    )
    mode_mapping = {
        "General Analysis": "analysis",
        "Comparative Analysis": "comparative",
        "Impact Forecast": "forecast"
    }
    st.session_state.analysis_mode = mode_mapping[analysis_mode]

    st.markdown('<h3 class="sidebar-header">📁 Chat Options</h3>', unsafe_allow_html=True)
    if st.button("🆕 New Chat"):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if st.session_state.messages:
            st.session_state.chat_history[timestamp] = st.session_state.messages
        st.session_state.messages = []
        st.rerun()

    st.markdown('<h3 class="sidebar-header">📜 Chat History</h3>', unsafe_allow_html=True)
    for k in reversed(list(st.session_state.chat_history.keys())):
        if st.button(k):
            st.session_state.messages = st.session_state.chat_history[k]
            st.rerun()
    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('<div class="sidebar-upload">', unsafe_allow_html=True)
    st.markdown('<h3 class="sidebar-header">📎 Upload Policy Documents</h3>', unsafe_allow_html=True)
    uploaded_file = st.file_uploader("Upload document to analyze", type=["txt", "pdf", "docx"])
    if uploaded_file:
        st.session_state.uploaded_file = uploaded_file
        st.markdown(f'<p style="color: white;">Uploaded: {uploaded_file.name}</p>', unsafe_allow_html=True)
        file_details = {
            "Filename": uploaded_file.name,
            "File size": f"{uploaded_file.size / 1024:.2f} KB"
        }
        st.markdown('<p style="color: white;">File Details:</p>', unsafe_allow_html=True)
        for key, value in file_details.items():
            st.markdown(f'<p style="color: white;">- <b>{key}:</b> {value}</p>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

# Chat area
chat_area = st.container()
with chat_area:
    st.markdown('<div class="chat-area">', unsafe_allow_html=True)
    for msg in st.session_state.messages:
        with st.chat_message(msg["role"]):
            st.markdown(f'<p style="color: white;">{msg["content"]}</p>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

# Example prompts
with st.expander("💡 Example prompts for your selected analysis mode"):
    if st.session_state.analysis_mode == "analysis":
        st.markdown("""
        - Analyze the impact of increasing minimum wage to $15/hour on small businesses
        - What are the implications of extending public transit to suburban areas?
        - Evaluate the effectiveness of tax incentives for renewable energy adoption
        """)
    elif st.session_state.analysis_mode == "comparative":
        st.markdown("""
        - Compare voucher-based and public housing approaches to affordable housing
        - Contrast carbon tax vs. cap-and-trade systems for reducing emissions
        - Compare merit-based vs. need-based scholarship programs for higher education
        """)
    else:
        st.markdown("""
        - Forecast the economic impact of implementing a four-day workweek
        - What would be the long-term effects of universal basic income?
        - Project the outcomes of reducing police department funding by 10%
        """)

# Chat input
if prompt := st.chat_input("Type your policy question..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.rerun()

# Process assistant response
if st.session_state.messages and st.session_state.messages[-1]["role"] == "user":
    with st.chat_message("assistant"):
        with st.spinner("Generating response..."):
            user_prompt = st.session_state.messages[-1]["content"]
            # Detect simple greetings
            greetings = ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]
            is_greeting = user_prompt.strip().lower() in greetings

            # Choose appropriate template
            if is_greeting:
                selected_template = prompt_templates["greeting"]
                max_tokens = 50
            else:
                selected_template = prompt_templates[st.session_state.analysis_mode]
                max_tokens = 300

            # Format prompt
            formatted_prompt = selected_template.format(query=user_prompt)

            # Append uploaded document context if any
            if st.session_state.uploaded_file and not is_greeting:
                try:
                    file_text = st.session_state.uploaded_file.getvalue().decode("utf-8")
                    formatted_prompt += f"\n\nPolicy document context:\n{file_text}"
                except Exception as e:
                    st.error(f"Error reading file: {e}")

            # Generate model output
            inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=max_tokens)
            reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Display & store
            st.markdown(f'<p style="color: white;">{reply}</p>', unsafe_allow_html=True)
            st.session_state.messages.append({"role": "assistant", "content": reply})


Writing my_file.py


In [ ]:
# Install necessary packages
!pip install streamlit
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Kill all existing tunnels
ngrok.kill()


In [ ]:

# Authenticate ngrok with your token
!ngrok authtoken " "  # Replace this with your actual ngrok token



# Run Streamlit app in the background and expose it via ngrok
from pyngrok import ngrok
import os

# Run Streamlit app (background execution)
os.system("streamlit run my_file.py &")

# Open ngrok tunnel for the Streamlit app (exposing port 8501)
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is running at: NgrokTunnel: "https://0766-34-143-135-233.ngrok-free.app" -> "http://localhost:8501"
